In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [3]:
!tar xf spark-3.3.1-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [6]:
import findspark
findspark.init()

In [7]:
findspark.find()

'/content/spark-3.3.1-bin-hadoop3'

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Thesis")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [5]:
spark.stop()

In [1]:
spark

In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.4 MB/s 
     |████████████████████████████████| 182 kB 65.0 MB/s 


In [11]:
# thu vien chinh sua cac loi unicode
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.2 MB/s 


In [12]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
import string    
import random
import json
import spacy
from spacy import displacy
from transformers import BertTokenizer, BertForTokenClassification
from transformers import TFRobertaModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-07 17:11:07--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-07 17:11:08--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-07 17:11:08--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [6]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import json
import os
import shutil

In [7]:
spark = sparknlp.start()

In [5]:
# connect ggdrive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [9]:

#save_model_address = '/content/drive/MyDrive/LUẬN VĂN-K18_CQ/02. Bình-Ngọc/Data/Dataset/conll/b_simp'
#save_model_address = '/content'

save_model_address = '/content/drive/MyDrive/Data Science/thesis/ML_NER/NERModel_config'

#save_model = BertForTokenClassification.from_pretrained(save_model_address, num_labels=20)
#tokenizer = BertTokenizer.from_pretrained(save_model_address,do_lower_case=True)

model = AutoModelForTokenClassification.from_pretrained(save_model_address)
tokenizer = AutoTokenizer.from_pretrained(save_model_address, do_lower_case=True, model_max_length=256)

#nlp = pipeline("ner", model=save_model, tokenizer=tokenizer, aggregation_strategy='simple',ignore_labels =['X','O'])


In [ ]:
loaded_ner_model = NerDLModel.load("/content/drive/MyDrive/Data Science/thesis/ML_NER/NERModel_config/pytorch_model")

In [2]:
df = spark.read.option('header','true').csv('/home/tyeson/Desktop/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True)
#df = spark.read.option('header','true').csv('/content/drive/MyDrive/Data Science/thesis/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True)

In [3]:
df

DataFrame[name: string, link: string, rating: string, enroll: string, instructor: string, time: string, levelrequirement: string, skillrequirement: string, SkillWillLearn: string, SkillGain: string, Subject: string, organization: string, fee: string, program: string, RelationInsOrg: string, Subtitle: string]

In [4]:
import pyspark.sql.functions as f
df = df.withColumn("enroll", f.regexp_replace(f.col("enroll"), ",", "").alias("enroll")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hours", "").alias("time")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hour", "").alias("time")) \
    .withColumn("fee", f.regexp_replace(f.col("fee"), "[a-zA-Z]+", "").alias("fee"))
    #.withColumn("fee", f.regexp_replace(f.col("fee"), "Enroll", "").alias("fee"))

In [5]:
from pyspark.sql.types import FloatType, LongType, IntegerType

df = df \
  .withColumn("rating" ,
              df["rating"]
              .cast(FloatType()))   \
  .withColumn("enroll",
              df["enroll"]
              .cast(LongType())) \
  .withColumn("time",df['time'].cast(IntegerType())) \
  .withColumn("fee", df['fee'].cast(IntegerType()))

In [6]:
df

DataFrame[name: string, link: string, rating: float, enroll: bigint, instructor: string, time: int, levelrequirement: string, skillrequirement: string, SkillWillLearn: string, SkillGain: string, Subject: string, organization: string, fee: int, program: string, RelationInsOrg: string, Subtitle: string]

In [7]:
from pyspark.sql.functions import col,when

In [8]:
df = df.na.fill(value=0, subset=['rating','enroll', 'fee','time'])

In [9]:
df = df.na.fill(value='None', subset=['link','instructor', 'levelrequirement', 'skillrequirement', 'SkillWillLearn','SkillGain', 'Subject', 'organization', 'program', 'RelationInsOrg', 'Subtitle'])

In [10]:
df = df.withColumn("levelrequirement", f.regexp_replace(f.col("levelrequirement"), "None", "Beginener").alias("levelrequirement")) \
    .withColumn("levelrequirement", f.regexp_replace(f.col("levelrequirement"), "Advanced", "Expert").alias("levelrequirement")) 


In [11]:
from pyspark.sql.functions import split, col, monotonically_increasing_id

df2 = df.select(col('name'),col('link'),col('rating'),col('enroll'),col('instructor'),col('time'),col('levelrequirement'),col('skillrequirement'),
                col('SkillWillLearn'),col('SkillGain'),col('Subject'),col('organization'),col('fee'),col('program'),col('RelationInsOrg'),split(col("Subtitle"),", ").alias("Subtitle"))
df2 = df2.withColumn("id", monotonically_increasing_id())

In [12]:
#df2 = df2.select(col('id'), col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'))
df2 = df2.withColumn("timestamp", f.current_timestamp()).alias("timestamp")
df2.show(5)

+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+---+--------------------+
|                name|                link|rating| enroll|          instructor|time|levelrequirement|    skillrequirement|      SkillWillLearn|           SkillGain|             Subject|        organization|fee|             program|      RelationInsOrg|            Subtitle| id|           timestamp|
+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+---+--------------------+
|    Machine Learning|https://www.cours...|   4.9|4491564|           Andrew Ng|  61|       Beginener|  

In [13]:
course = df2.select(col('id'), col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'), col('timestamp'))
course.show()

+---+--------------------+------+-------+----+---+--------------------+--------------------+
| id|                name|rating| enroll|time|fee|                link|           timestamp|
+---+--------------------+------+-------+----+---+--------------------+--------------------+
|  0|    Machine Learning|   4.9|4491564|  61|  0|https://www.cours...|2023-01-03 13:38:...|
|  1|Data Visualizatio...|   4.9|  12564|  18|  0|https://www.cours...|2023-01-03 13:38:...|
|  2|Spatial Analysis ...|   4.9|  18321|  14|  0|https://www.cours...|2023-01-03 13:38:...|
|  3|Python Data Struc...|   4.9| 800203|  19|  0|https://www.cours...|2023-01-03 13:38:...|
|  4|Neural Networks a...|   4.9|1013093|  27|  0|https://www.cours...|2023-01-03 13:38:...|
|  5|What is Data Scie...|   4.7| 516658|   9|  0|https://www.cours...|2023-01-03 13:38:...|
|  6|         Web of Data|   4.1|   4104|  18|  0|https://www.cours...|2023-01-03 13:38:...|
|  7|Share Data Throug...|   4.5|  68559|  24|  0|https://www.cours...

In [15]:
#df2 = df2.select(col('id'), col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'))
#df2  = course_data.withColumn("timestamp", f.current_timestamp()).alias("timestamp")

#df2 = df2.select(col('id'), col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'), col('timestamp'))

#query = "merge (course:Course{courseIdentifyKey: event.id , courseName: event.name ,courseEnrolled:[event.enroll], courseRating: event.rating, courseTime: event.time,courseFee: event.fee, courseUpdateDate: [event.timestamp], courselink: event.link})"
#query = "merge (course:Course{courseIdentifyKey: event.id, courseName: event.name,courseEnrolled:[event.enroll], courseUpdateDate: event.timestamp, courseTime: event.time})"
course.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("node.keys", "id")\
    .option("labels",":Cousse")\
    .save()

In [16]:
#ourse_data = df2.select(col('id'), col('name'), col('rating'), col('enroll'), col('time'), col('fee'), col('link'))
#course_data_time = course_data.withColumn("timestamp", f.current_timestamp()).alias("timestamp")
course_data_time.show(300)

+---+--------------------+------+-------+----+---+--------------------+--------------------+
| id|                name|rating| enroll|time|fee|                link|           timestamp|
+---+--------------------+------+-------+----+---+--------------------+--------------------+
|  0|    Machine Learning|   4.9|4491564|  61|  0|https://www.cours...|2022-12-09 01:33:...|
|  1|Data Visualizatio...|   4.9|  12564|  18|  0|https://www.cours...|2022-12-09 01:33:...|
|  2|Spatial Analysis ...|   4.9|  18321|  14|  0|https://www.cours...|2022-12-09 01:33:...|
|  3|Python Data Struc...|   4.9| 800203|  19|  0|https://www.cours...|2022-12-09 01:33:...|
|  4|Neural Networks a...|   4.9|1013093|  27|  0|https://www.cours...|2022-12-09 01:33:...|
|  5|What is Data Scie...|   4.7| 516658|   9|  0|https://www.cours...|2022-12-09 01:33:...|
|  6|         Web of Data|   4.1|   4104|  18|  0|https://www.cours...|2022-12-09 01:33:...|
|  7|Share Data Throug...|   4.5|  68559|  24|  0|https://www.cours...

In [14]:
df2 = df2.withColumn("instructor", f.regexp_replace(f.col("instructor"), ", PhD", "").alias("instructor")) \
    .withColumn("instructor", f.regexp_replace(f.col("instructor"), ", Ph.D.", "").alias("instructor")) \
    .withColumn("instructor", f.regexp_replace(f.col("instructor"), chr(34), "").alias("instructor"))

In [15]:
df2.show()

+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+---+--------------------+
|                name|                link|rating| enroll|          instructor|time|levelrequirement|    skillrequirement|      SkillWillLearn|           SkillGain|             Subject|        organization|fee|             program|      RelationInsOrg|            Subtitle| id|           timestamp|
+--------------------+--------------------+------+-------+--------------------+----+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+--------------------+---+--------------------+
|    Machine Learning|https://www.cours...|   4.9|4491564|           Andrew Ng|  61|       Beginener|  

In [23]:
instructor = df2.select(col("id"), split(col("instructor"),",").alias("instructor"))
instructor.show()

+---+--------------------+
| id|          instructor|
+---+--------------------+
|  0|         [Andrew Ng]|
|  1|[Nicky Bull, Dr P...|
|  2|         [Don Boyes]|
|  3|[Charles Russell ...|
|  4|[Andrew Ng, Kian ...|
|  5|[Rav Ahuja, Alex ...|
|  6|[Catherine Faron ...|
|  7|[Google Career Ce...|
|  8|       [Margaret Ng]|
|  9|       [Nicole Ball]|
| 10|[Brenda Gunderson...|
| 11|[Jessen Hobson, R...|
| 12|[Carrie Wright, S...|
| 13|[Carrie Wright, S...|
| 14|[Carrie Wright, S...|
| 15|[Carrie Wright, S...|
| 16|[Hadi H. K. Kharr...|
| 17|   [Youngju Nielsen]|
| 18|        [Beth Simon]|
| 19|[Frauke Kreuter, ...|
+---+--------------------+
only showing top 20 rows



In [24]:
from pyspark.sql.functions import explode
instructor = instructor.select(instructor.id, explode("instructor").alias("instructor"))
instructor.count()

487

In [25]:
instructorNode = instructor.select(instructor.instructor)
instructor.show()

+---+--------------------+
| id|          instructor|
+---+--------------------+
|  0|           Andrew Ng|
|  1|          Nicky Bull|
|  1|Dr Prashan S. M. ...|
|  2|           Don Boyes|
|  3|Charles Russell S...|
|  4|           Andrew Ng|
|  4|   Kian Katanforoosh|
|  4|Younes Bensouda M...|
|  5|           Rav Ahuja|
|  5|         Alex Aklson|
|  6|Catherine Faron Z...|
|  6|       Fabien Gandon|
|  6|       Olivier Corby|
|  7|Google Career Cer...|
|  8|         Margaret Ng|
|  9|         Nicole Ball|
| 10|    Brenda Gunderson|
| 10|       Brady T. West|
| 10|       Kerby Shedden|
| 11|       Jessen Hobson|
+---+--------------------+
only showing top 20 rows



In [26]:
instructorNode.count()

487

In [23]:
#query = "merge (course:Course{courseIdentifyKey: event.id , courseName: event.name ,courseEnrolled:[event.enroll], courseRating: event.rating, courseTime: event.time,courseFee: event.fee, courseUpdateDate: [event.timestamp], courselink: event.link})"
#query = "merge (course:Course{courseIdentifyKey: event.id, courseName: event.name,courseEnrolled:[event.enroll], courseUpdateDate: event.timestamp, courseTime: event.time})"
instructorNode.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("node.keys", "instructor")\
    .option("labels",":Instructor")\
    .save()

In [32]:
instructor.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("relationship", 'TEACH_BY')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','source:instructor')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':instructor')\
    .option('relationship.target.node.keys','target:instructor')\
    .option('relationship.target.save.mode','Match')\
    .save()

23/01/03 15:30:47 ERROR Utils: Aborting task
java.lang.IllegalArgumentException: Write failed due to the following errors:
 - Schema is missing target from option `relationship.target.node.keys`
 - Schema is missing source from option `relationship.source.node.keys`

The option key and value might be inverted.
	at org.neo4j.spark.util.ValidateSchemaOptions.validate(Validations.scala:38)
	at org.neo4j.spark.util.Validations$.$anonfun$validate$1(Validations.scala:12)
	at org.neo4j.spark.util.Validations$.$anonfun$validate$1$adapted(Validations.scala:12)
	at scala.collection.immutable.Set$Set1.foreach(Set.scala:141)
	at org.neo4j.spark.util.Validations$.validate(Validations.scala:12)
	at org.neo4j.spark.service.Neo4jWriteMappingStrategy.relationship(MappingService.scala:92)
	at org.neo4j.spark.service.Neo4jWriteMappingStrategy.relationship(MappingService.scala:18)
	at org.neo4j.spark.service.MappingService.convert(MappingService.scala:235)
	at org.neo4j.spark.writer.BaseDataWriter.write(B

Py4JJavaError: An error occurred while calling o299.save.
: org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:749)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:409)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:353)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.writeWithV2(WriteToDataSourceV2Exec.scala:262)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:332)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:331)
	at org.apache.spark.sql.execution.datasources.v2.OverwriteByExpressionExec.run(WriteToDataSourceV2Exec.scala:262)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(

In [27]:
instructor.select(instructor.id,instructor.instructor).show(20)

+---+--------------------+
| id|          instructor|
+---+--------------------+
|  0|           Andrew Ng|
|  1|          Nicky Bull|
|  1|Dr Prashan S. M. ...|
|  2|           Don Boyes|
|  3|Charles Russell S...|
|  4|           Andrew Ng|
|  4|   Kian Katanforoosh|
|  4|Younes Bensouda M...|
|  5|           Rav Ahuja|
|  5|         Alex Aklson|
|  6|Catherine Faron Z...|
|  6|       Fabien Gandon|
|  6|       Olivier Corby|
|  7|Google Career Cer...|
|  8|         Margaret Ng|
|  9|         Nicole Ball|
| 10|    Brenda Gunderson|
| 10|       Brady T. West|
| 10|       Kerby Shedden|
| 11|       Jessen Hobson|
+---+--------------------+
only showing top 20 rows



In [ ]:
instructor = df2.select(col("id"), col('name'),col('link'),col('rating'),col('enroll'),split(col("instructor"),",").alias("instructor"),col('time'),col('levelrequirement'),col('skillrequirement'),
                col('SkillWillLearn'),col('SkillGain'),col('Subject'),col('organization'),col('fee'),col('program'),col('RelationInsOrg'),col("Subtitle"),col('timestamp'))

In [ ]:
level = df2.select(col("id"), col('levelrequirement'))
level.show()

In [ ]:
level.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("node.keys", "level")\
    .option("labels",":Level")\
    .save()

In [ ]:
subtitle = df2.select(col("id"), split(col("Subtitle"),",").alias("Subtitle"))
subtitle.show()

In [ ]:
subtitle = subtitle.select(subtitle.id, explode("Subtitle").alias("language"))
subtitle.show()

In [ ]:
subtitle.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("node.keys", "language")\
    .option("labels",":Language")\
    .save()

In [ ]:
level.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("relationship", 'HAS_LEVEL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','source:level')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Level')\
    .option('relationship.target.node.keys','target:level')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [ ]:
instructor.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("relationship", 'TEACH_BY')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','source:instructor')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':instructor')\
    .option('relationship.target.node.keys','target:instructor')\
    .option('relationship.target.save.mode','Match')\
    .save()